In [ ]:
import pandas as pd

In [ ]:
train_asp = pd.read_csv(
    'dev_pred_aspects_2.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id','text'])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
result = pd.merge(train_asp, train_texts, on="text_id", how="outer")

In [ ]:
result = result.loc[result['sentiment'] != 'both']

In [ ]:
def get_context(text, start,	end,	left_win, right_win):
  start_index, end_index = word_index(text, start, end)
  text_split = text.split()
  result = []
  for i in range(left_win):
    if start_index - (left_win - i + 1) > 0:
      result.append(text_split[start_index - (left_win - i + 1)])
  result.append(text[start:end])
  for i in range(right_win):
    if end_index + i < len(text_split):
      result.append(text_split[end_index + i])
  return result

In [ ]:
kartaslov = pd.read_csv('kartaslovsent_4.csv', sep=';')

In [ ]:
!pip install -U pip setuptools wheel

!pip install -U spacy

!python -m spacy download ru_core_news_lg

     |████████████████████████████████| 514.5 MB 8.7 kB/s             
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
import spacy
nlp = spacy.load('ru_core_news_lg')
tok = nlp.tokenizer

Функция, получающая индекс слова в тексте по значению его первого и последнего символов.

In [ ]:
import string 
def index_of_word(text, token, start):
    tokenized = tok(text)
    ngram = tok(token)
    if len(ngram) > 1:
        token = ngram[0].text
    shift = len(ngram) - 1
    new_start = 0
    for i, t in enumerate(tokenized):
        t = t.text
        if t == token:
            if text.index(t, new_start) == start:
                return i + shift
            else:
                new_start = (text.index(t, new_start) + len(token))

Функция, получающая некоторые зависимые (в основном для именной группы) с помощью синтаксического анализа spaCy.

In [ ]:
def get_nps(text, start, end):
  first_w = index_of_word(text, text[start:end], start)
  doc = nlp(text)
  children_second_level = []
  i = doc[first_w]
  for child in i.lefts:
    if child.dep_ == 'amod' or child.dep_ == 'acl' or child.dep_ == 'advmod':
      for x in child.lefts:
        if x.dep_ != 'nsubj':
          if x.lefts:
            for y in x.lefts:
              children_second_level.append(y.text)
          children_second_level.append(x.text)
      children_second_level.append(child.text)
  children_second_level.append(i.text)
  for child in i.rights:
    if child.dep_ == 'nmod':
      children_second_level.append(child.text)
  for x in i.head.head.children:
    if x.dep_ == 'advmod':
      children_second_level.append(x.text)
  return children_second_level

Функция, получающая вершину и некоторые зависимые вершины искомого слова-аспекта.

In [ ]:
def get_pred_adj(text, start, end):
  first_w = index_of_word(text, text[start:end], start)
  doc = nlp(text)
  st_pred = []
  i = doc[first_w]
  if i.dep_ == 'nsubj':
    st_pred.append(i.text)
    for dep in i.head.lefts:
      if dep.dep_ not in ['cc', 'punct', 'conj']:
        if dep.lefts:
          for x in dep.lefts:
            if x.dep_ not in ['cc', 'punct']:
              st_pred.append(x.text)
        if dep.text != i.text:
          if dep.dep_ not in ['cc', 'punct']:
            st_pred.append(dep.text)
        if dep.rights:
          for x in dep.rights:
            if x.dep_ not in ['cc', 'punct']:
              st_pred.append(x.text)
    st_pred.append(i.head.text)
    for dep in i.head.rights:
      if dep.dep_ not in ['cc', 'punct', 'conj']:
        if dep.lefts:
          for x in dep.lefts:
            if x.dep_ not in ['cc', 'punct']:
              st_pred.append(x.text)
        if dep.text != i.text:
          if dep.dep_ not in ['cc', 'punct', 'conj']:
            st_pred.append(dep.text)
        if dep.rights:
          for x in dep.rights:
            if x.dep_ not in ['cc', 'punct', 'conj']:
              st_pred.append(x.text)
  return st_pred

Функция, возвращающая группы аспекта, его зависимые, либо, если таковых не нашлось, вершину.

In [ ]:
def get_groups(text, start, end):
  try:
    group = get_nps(text, start, end)
    if len(group) < 2:
      group_1 = get_pred_adj(text, start, end)
      if group_1 == []:
        return word_tokenize(text)
      else:
        return group_1
    else:
      return group
  except:
    return word_tokenize(text)

Обрабатываем тестовые данные (аспекты, выделенные на предыдущем этапе).

In [ ]:
result['parts'] = result.progress_apply(lambda row: get_groups(row['text'], row['start'], row['end']), axis=1)

100%|██████████| 1405/1405 [02:20<00:00,  9.97it/s]


Открываем доработанный нами тональный словарь.

In [ ]:
karta_dict = kartaslov.set_index('term')['tag'].to_dict()

In [ ]:
pip install pymorphy2

In [ ]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Получаем сентимент для каждой группы (сумма сентиментов каждого слова, при отрицательной тональности -1, при положительной 1, при нейтральной 0.

In [ ]:
def get_sentiment(list_words):
  sent_num = 0
  for word in list_words:
    word = morph.parse(word.lower())[0].normal_form
    if word in karta_dict.keys():
      if karta_dict[word] == 'PSTV':
        sent_num += 1
      elif karta_dict[word] == 'NGTV':
        sent_num -= 1
  if sent_num < 0:
    return 2
  if sent_num > 0:
    return 1
  if sent_num == 0:
    return 0

In [ ]:
result['sentiment'] = result['parts'].progress_apply(get_sentiment)

100%|██████████| 1405/1405 [00:23<00:00, 60.05it/s]


Приводим файл в необходимый для подсчета качества вид и скачиваем его.

In [ ]:
del result['parts']
del result['text']

In [ ]:
def make_val(number):
  if number == 0:
    return 'neutral'
  if number == 1:
    return 'positive'
  if number == 2:
    return 'negative'

In [ ]:
result['sentiment'] = result['sentiment'].apply(make_val)

In [ ]:
result.to_csv('test.txt', sep='\t', header=None, index=False)

Считаем качество.

In [ ]:
gold_test_path = "dev_aspects.txt"
pred_test_path = "test.txt"

In [ ]:
from collections import defaultdict

In [ ]:
gold_aspect_cats = {}
with open(gold_test_path) as fg:
    for line in fg:
        line = line.rstrip('\r\n').split('\t')
        if line[0] not in gold_aspect_cats:
            gold_aspect_cats[line[0]] = {"starts":[], "ends":[], "cats":[], "sents":[]}
        gold_aspect_cats[line[0]]["starts"].append(int(line[3]))
        gold_aspect_cats[line[0]]["ends"].append(int(line[4]))
        gold_aspect_cats[line[0]]["cats"].append(line[1])
        gold_aspect_cats[line[0]]["sents"].append(line[5])

In [ ]:
full_match, partial_match, full_cat_match, partial_cat_match = 0, 0, 0, 0
total = 0
fully_matched_pairs = []
partially_matched_pairs = []
with open(pred_test_path) as fp:
    for line in fp:    
        total += 1
        line = line.rstrip('\r\n').split('\t')
        start, end = int(line[3]), int(line[4])
        category = line[1]
        doc_gold_aspect_cats = gold_aspect_cats[line[0]]
        if start in doc_gold_aspect_cats["starts"]:
            i = doc_gold_aspect_cats["starts"].index(start)
            if doc_gold_aspect_cats["ends"][i] == end:
                full_match += 1
                if doc_gold_aspect_cats["cats"][i] == category:
                    full_cat_match += 1
                else:
                    partial_cat_match += 1
                fully_matched_pairs.append(
                    (
                        [
                            doc_gold_aspect_cats["starts"][i], 
                            doc_gold_aspect_cats["ends"][i], 
                            doc_gold_aspect_cats["cats"][i],
                            doc_gold_aspect_cats["sents"][i]
                        ],
                        line
                    )
                )
                continue
        for s_pos in doc_gold_aspect_cats["starts"]:
            if start <= s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if doc_gold_aspect_cats["ends"][i] == end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    continue
                matched = False
                for e_pos in doc_gold_aspect_cats["ends"][i:]:
                    if s_pos <= end <= e_pos:
                        partial_match += 1
                        partially_matched_pairs.append(
                            (
                                [
                                    doc_gold_aspect_cats["starts"][i], 
                                    doc_gold_aspect_cats["ends"][i], 
                                    doc_gold_aspect_cats["cats"][i],
                                    doc_gold_aspect_cats["sents"][i]
                                ],
                                line
                            )
                        )
                        if doc_gold_aspect_cats["cats"][i] == category:
                            partial_cat_match += 1
                        matched = True
                        break
                if matched:
                    break
            if start > s_pos:
                i = doc_gold_aspect_cats["starts"].index(s_pos)
                if start < doc_gold_aspect_cats["ends"][i] <= end:
                    partial_match += 1
                    partially_matched_pairs.append(
                        (
                            [
                                doc_gold_aspect_cats["starts"][i], 
                                doc_gold_aspect_cats["ends"][i], 
                                doc_gold_aspect_cats["cats"][i],
                                doc_gold_aspect_cats["sents"][i]
                            ],
                            line
                        )
                    )
                    if doc_gold_aspect_cats["cats"][i] == category:
                        partial_cat_match += 1
                    break

In [ ]:
gold_size = sum([len(gold_aspect_cats[x]["cats"]) for x in gold_aspect_cats])

In [ ]:
def sentiment_accuracy(matches):
    matched_sentiment = 0.
    for pair in matches:
        *_, gold_s = pair[0]
        *_, pred_s = pair[1]
        if gold_s == pred_s:
            matched_sentiment += 1
    print(f"Mention sentiment accuracy: {matched_sentiment / len(matches)}")

In [ ]:
sentiment_accuracy(fully_matched_pairs)

Mention sentiment accuracy: 0.5313432835820896


In [ ]:
sentiment_accuracy(partially_matched_pairs)

Mention sentiment accuracy: 0.5034013605442177
